# Build Standard Bars(Time Bar, Tick Bar, Volume Bar and Dollar Bar)

In [1]:
import gc
import glob
import os
from operator import itemgetter
from typing import Dict, List, Tuple

import dask.bag as db
import numpy as np
import pandas as pd
from dask.diagnostics import ProgressBar

## Build on a single CSV file

In [2]:
def build_standard_bars(input_csv_file: str, bar_type:str, bar_size: np.int64, output_csv_file: str)->None:
    assert bar_type == 'TimeBar' or bar_type == 'TickBar' or bar_type == 'VolumeBar' or bar_type == 'DollarBar'
    df = pd.read_csv(input_csv_file, engine='c',
                     dtype={'exchange': 'category', 'marketType': 'category', 'pair': 'category',
                            'timestamp': 'int64', 'price': 'float32',
                            'quantity': 'float32', 'side': 'bool',
                            'trade_id': 'string' if 'BitMEX' in input_csv_file else 'int64'})
    if 'BitMEX' in input_csv_file:
        assert df['timestamp'].is_monotonic_increasing
    else:
        assert df['trade_id'].is_monotonic_increasing

    df['quantity_sell'] = df['quantity'] * df['side']
    df['quantity_buy'] = df['quantity']-df['quantity_sell']
    df['quantity_quote'] = df['quantity'] * df['price']
    df['quantity_quote_sell'] = df['quantity_quote'] * df['side']
    df['quantity_quote_buy'] = df['quantity_quote']-df['quantity_sell']
    df['count'] = 1
    df['count_sell'] = df['side'].astype('int32')
    df['count_buy'] = df['count']-df['count_sell']
    
    if bar_type == 'TimeBar':
        df['bar_index'] = df['timestamp'] // bar_size
    elif bar_type == 'TickBar':
        df['bar_index'] = (df.index // bar_size).to_series().reset_index(drop=True)
#     elif bar_type == 'VolumeBar':
#         df['bar_index'] = df['quantity'].astype('float64').cumsum().floordiv(bar_size).astype('uint64')
#     elif bar_type == 'DollarBar':
#         df['bar_index'] = df['quantity_quote'].astype('float64').cumsum().floordiv(bar_size).astype('uint64')
    elif bar_type == 'VolumeBar' or bar_type == 'DollarBar':
        df['bar_index'] = -1
        bar_index = 0
        volume_sum = 0
        for row in df.itertuples():
            df.at[row.Index, 'bar_index'] = bar_index
            volume_sum += row.quantity if bar_type == 'VolumeBar' else row.quantity_quote
            if volume_sum >= bar_size:
                volume_sum = 0
                bar_index += 1
    else:
        raise ValueError(f'Unknown bar_type: {bar_type}')

    assert df['bar_index'].is_monotonic_increasing
    assert pd.Series.all(df['bar_index'] >= 0)

    bars_df = df.groupby('bar_index').agg({
            'timestamp': ['last', 'first'],
            'price': ['first', 'max', 'min', 'last', 'mean', 'median'],
            'quantity': 'sum',
            'quantity_sell': 'sum',
            'quantity_buy': 'sum',
            'quantity_quote': 'sum',
            'quantity_quote_sell': 'sum',
            'quantity_quote_buy': 'sum',
            'count': 'sum',
            'count_sell': 'sum',
            'count_buy': 'sum'
        }
    )
    
    column_names_map = {
        'timestamp_first': 'timestamp_start',
        'timestamp_last': 'timestamp',
        'price_first': 'open',
        'price_max': 'high',
        'price_min': 'low',
        'price_last': 'close',
        'price_mean': 'mean',
        'price_median': 'median',
        'quantity_sum': 'volume',
        'quantity_sell_sum': 'volume_sell',
        'quantity_buy_sum': 'volume_buy',
        'quantity_quote_sum': 'volume_quote',
        'quantity_quote_sell_sum': 'volume_quote_sell',
        'quantity_quote_buy_sum': 'volume_quote_buy',
        'count_sum': 'count',
        'count_sell_sum': 'count_sell',
        'count_buy_sum': 'count_buy'
    }
    # see https://stackoverflow.com/a/14508355/381712
    new_columns_names = [column_names_map['_'.join(col).strip()] for col in bars_df.columns.values] 
    bars_df.columns = new_columns_names

    if bar_type == 'TimeBar':
        bars_df['timestamp'] = (bars_df['timestamp'] // bar_size + 1) * bar_size
        # bars_df['timestamp_start'] = bars_df['timestamp'] - bar_size

    exchange, market_type, pair, _ = os.path.basename(input_csv_file).split('.')
    bars_df['exchange'] = exchange
    bars_df['market_type'] = market_type
    bars_df['pair'] = pair
    bars_df['bar_type'] = bar_type
    bars_df['bar_size'] = bar_size
    # sort columns
    bars_df = bars_df[['exchange', 'market_type', 'pair', 'bar_type', 'bar_size'] + new_columns_names]

    os.makedirs(os.path.dirname(output_csv_file), exist_ok=True)
    bars_df.to_csv(output_csv_file, index=False)

    # optional
    del bars_df
    del df
    gc.collect()

In [3]:
CSV_FILE = '/data/merged_csv/OKEx.Swap.XMR_USDT.csv'

In [4]:
total_volume = pd.read_csv(CSV_FILE, usecols=['quantity']).sum()

In [5]:
build_standard_bars(CSV_FILE, 'TimeBar', 10000, '/data/bars/TimeBar/10000/TimeBar.10000.OKEx.Swap.XMR_USDT.csv')

In [6]:
time_bars = pd.read_csv('/data/bars/TimeBar/10000/TimeBar.10000.OKEx.Swap.XMR_USDT.csv')

In [7]:
time_bars.head()

,exchange,market_type,pair,bar_type,bar_size,timestamp,timestamp_start,open,high,low,...,median,volume,volume_sell,volume_buy,volume_quote,volume_quote_sell,volume_quote_buy,count,count_sell,count_buy
0,OKEx,Swap,XMR_USDT,TimeBar,10000,1590216470000,1590216461309,63.42,63.42,63.42,...,63.42,2.8,2.8,0.0,177.575990,177.575990,174.775990,1,1,0
1,OKEx,Swap,XMR_USDT,TimeBar,10000,1590220820000,1590220818421,63.56,63.56,63.56,...,63.56,0.7,0.0,0.7,44.492000,0.000000,44.492000,1,0,1
2,OKEx,Swap,XMR_USDT,TimeBar,10000,1590221710000,1590221708996,63.07,63.07,63.07,...,63.07,0.6,0.6,0.0,37.842003,37.842003,37.242004,1,1,0
3,OKEx,Swap,XMR_USDT,TimeBar,10000,1590229380000,1590229378881,62.41,62.41,62.41,...,62.41,1.5,1.5,0.0,93.615000,93.615000,92.115000,1,1,0
4,OKEx,Swap,XMR_USDT,TimeBar,10000,1590249620000,1590249619582,63.49,63.49,63.49,...,63.49,0.5,0.0,0.5,31.745000,0.000000,31.745000,1,0,1


In [8]:
time_bars['volume'].sum()/total_volume

quantity    1.0
dtype: float64

In [9]:
build_standard_bars(CSV_FILE, 'TickBar', 4, '/data/bars/TickBar/4/TickBar.4.OKEx.Swap.XMR_USDT.csv')

In [10]:
tick_bars = pd.read_csv('/data/bars/TickBar/4/TickBar.4.OKEx.Swap.XMR_USDT.csv')

In [11]:
tick_bars.head()

,exchange,market_type,pair,bar_type,bar_size,timestamp,timestamp_start,open,high,low,...,median,volume,volume_sell,volume_buy,volume_quote,volume_quote_sell,volume_quote_buy,count,count_sell,count_buy
0,OKEx,Swap,XMR_USDT,TickBar,4,1590229378881,1590216461309,63.42,63.56,62.41,...,63.245,5.6,4.9,0.7,353.52500,309.033,348.62500,4,3,1
1,OKEx,Swap,XMR_USDT,TickBar,4,1590263779094,1590249619582,63.49,64.39,63.49,...,64.090,1.4,0.0,1.4,89.54600,0.000,89.54600,4,0,4
2,OKEx,Swap,XMR_USDT,TickBar,4,1590292415864,1590265720050,64.69,64.99,63.22,...,64.760,5.7,0.7,5.0,368.41200,44.254,367.71198,4,1,3
3,OKEx,Swap,XMR_USDT,TickBar,4,1590293777431,1590293593504,63.22,63.28,63.22,...,63.250,8.0,0.0,8.0,506.00003,0.000,506.00003,4,0,4
4,OKEx,Swap,XMR_USDT,TickBar,4,1590295691153,1590295679230,63.03,63.03,63.02,...,63.020,4.5,0.0,4.5,283.60600,0.000,283.60600,4,0,4


In [12]:
tick_bars['volume'].sum()/total_volume

quantity    1.0
dtype: float64

In [13]:
build_standard_bars(CSV_FILE, 'VolumeBar', 10, '/data/bars/VolumeBar/10/VolumeBar.10.OKEx.Swap.XMR_USDT.csv')

In [14]:
volume_bars = pd.read_csv('/data/bars/VolumeBar/10/VolumeBar.10.OKEx.Swap.XMR_USDT.csv')

In [15]:
volume_bars.head()

,exchange,market_type,pair,bar_type,bar_size,timestamp,timestamp_start,open,high,low,...,median,volume,volume_sell,volume_buy,volume_quote,volume_quote_sell,volume_quote_buy,count,count_sell,count_buy
0,OKEx,Swap,XMR_USDT,VolumeBar,10,1590265720050,1590216461309,63.42,64.83,62.41,...,63.824997,11.599999,4.9,6.7,741.233,309.033,736.33295,10,3,7
1,OKEx,Swap,XMR_USDT,VolumeBar,10,1590295679230,1590267970755,64.99,64.99,63.03,...,63.220000,10.700000,0.7,10.0,677.098,44.254,676.39800,7,1,6
2,OKEx,Swap,XMR_USDT,VolumeBar,10,1590298286064,1590295685165,63.02,63.04,62.94,...,63.020000,10.400000,0.0,10.4,655.206,0.000,655.20600,8,0,8
3,OKEx,Swap,XMR_USDT,VolumeBar,10,1590298566867,1590298287629,63.04,63.07,63.04,...,63.055000,11.400000,0.0,11.4,718.871,0.000,718.87100,8,0,8
4,OKEx,Swap,XMR_USDT,VolumeBar,10,1590298817240,1590298729120,63.08,63.08,63.07,...,63.070000,12.000000,0.0,12.0,756.872,0.000,756.87200,3,0,3


In [16]:
volume_bars['volume'].sum()/total_volume

quantity    1.0
dtype: float64

In [17]:
build_standard_bars(CSV_FILE, 'DollarBar', 1000, '/data/bars/DollarBar/1000/DollarBar.1000.OKEx.Swap.XMR_USDT.csv')

In [18]:
dollar_bars = pd.read_csv('/data/bars/DollarBar/1000/DollarBar.1000.OKEx.Swap.XMR_USDT.csv')

In [19]:
dollar_bars.head()

,exchange,market_type,pair,bar_type,bar_size,timestamp,timestamp_start,open,high,low,...,median,volume,volume_sell,volume_buy,volume_quote,volume_quote_sell,volume_quote_buy,count,count_sell,count_buy
0,OKEx,Swap,XMR_USDT,DollarBar,1000,1590293593507,1590216461309,63.42,64.99,62.41,...,63.525000,16.700000,5.6,11.1,1064.363,353.28700,1058.763,14,4,10
1,OKEx,Swap,XMR_USDT,DollarBar,1000,1590298286064,1590293777429,63.28,63.28,62.94,...,63.020000,16.000000,0.0,16.0,1009.174,0.00000,1009.174,11,0,11
2,OKEx,Swap,XMR_USDT,DollarBar,1000,1590298814347,1590298287629,63.04,63.08,63.04,...,63.065002,20.200000,0.0,20.2,1273.919,0.00000,1273.919,10,0,10
3,OKEx,Swap,XMR_USDT,DollarBar,1000,1590299007467,1590298817240,63.07,63.09,63.07,...,63.070000,16.700000,0.0,16.7,1053.363,0.00000,1053.363,7,0,7
4,OKEx,Swap,XMR_USDT,DollarBar,1000,1590336260051,1590299009625,63.09,63.69,62.45,...,63.150000,15.900001,8.2,7.7,1001.723,515.58105,993.523,20,11,9


In [20]:
dollar_bars['volume'].sum()/total_volume

quantity    1.0
dtype: float64

In [21]:
os.remove('/data/bars/TimeBar/10000/TimeBar.10000.OKEx.Swap.XMR_USDT.csv')
os.remove('/data/bars/TickBar/4/TickBar.4.OKEx.Swap.XMR_USDT.csv')
os.remove('/data/bars/VolumeBar/10/VolumeBar.10.OKEx.Swap.XMR_USDT.csv')
os.remove('/data/bars/DollarBar/1000/DollarBar.1000.OKEx.Swap.XMR_USDT.csv')

## Build on multiple CSV files

In [22]:
def generate_tasks(csv_files: List[str], bar_type: str, bar_sizes: List[int], output_dir: str)->List[Tuple[str, str, int, str]]:
    tasks = [(file, bar_type, bar_size, os.path.join(output_dir, str(bar_size),f'{bar_type}.{bar_size}.{os.path.basename(file)}'))
             for file in csv_files for bar_size in bar_sizes]
    return tasks

In [23]:
def run_tasks_parallell(tasks: List[Tuple[str, str, int, str]])->None:
    with ProgressBar():
        db.from_sequence(tasks).map(lambda t: build_standard_bars(t[0], t[1], t[2], t[3])).compute()

In [24]:
btc_files = glob.glob('/data/merged_csv/*BTC_USD*.csv')
eth_files = glob.glob('/data/merged_csv/*ETH_USD*.csv')

In [25]:
btc_files

['/data/merged_csv/Newdex.Spot.BTC_USDT.csv',
 '/data/merged_csv/OKEx.Spot.BTC_USDT.csv',
 '/data/merged_csv/Binance.Swap.BTC_USDT.csv',
 '/data/merged_csv/Kraken.Spot.BTC_USDT.csv',
 '/data/merged_csv/Bitfinex.Swap.BTC_USDT.csv',
 '/data/merged_csv/Bitstamp.Spot.BTC_USD.csv',
 '/data/merged_csv/Huobi.Spot.BTC_USDT.csv',
 '/data/merged_csv/OKEx.Swap.BTC_USDT.csv',
 '/data/merged_csv/OKEx.Swap.BTC_USD.csv',
 '/data/merged_csv/Kraken.Spot.BTC_USD.csv',
 '/data/merged_csv/WhaleEx.Spot.BTC_USDT.csv',
 '/data/merged_csv/Bitfinex.Spot.BTC_USD.csv',
 '/data/merged_csv/MXC.Spot.BTC_USDT.csv',
 '/data/merged_csv/BitMEX.Swap.BTC_USD.csv',
 '/data/merged_csv/Binance.Spot.BTC_USDT.csv',
 '/data/merged_csv/CoinbasePro.Spot.BTC_USD.csv',
 '/data/merged_csv/Huobi.Swap.BTC_USD.csv',
 '/data/merged_csv/Bitfinex.Spot.BTC_USDT.csv']

In [26]:
eth_files

['/data/merged_csv/Huobi.Swap.ETH_USD.csv',
 '/data/merged_csv/Kraken.Spot.ETH_USD.csv',
 '/data/merged_csv/Bitfinex.Spot.ETH_USD.csv',
 '/data/merged_csv/Bitstamp.Spot.ETH_USD.csv',
 '/data/merged_csv/MXC.Spot.ETH_USDT.csv',
 '/data/merged_csv/CoinbasePro.Spot.ETH_USD.csv',
 '/data/merged_csv/Bitfinex.Swap.ETH_USDT.csv',
 '/data/merged_csv/Newdex.Spot.ETH_USDT.csv',
 '/data/merged_csv/WhaleEx.Spot.ETH_USDT.csv',
 '/data/merged_csv/Kraken.Spot.ETH_USDT.csv',
 '/data/merged_csv/OKEx.Swap.ETH_USDT.csv',
 '/data/merged_csv/BitMEX.Swap.ETH_USD.csv',
 '/data/merged_csv/OKEx.Swap.ETH_USD.csv',
 '/data/merged_csv/OKEx.Spot.ETH_USDT.csv',
 '/data/merged_csv/Binance.Swap.ETH_USDT.csv',
 '/data/merged_csv/Bitfinex.Spot.ETH_USDT.csv',
 '/data/merged_csv/Binance.Spot.ETH_USDT.csv',
 '/data/merged_csv/Huobi.Spot.ETH_USDT.csv']

In [32]:
!ls -lhS /data/merged_csv/*BTC_USD*.csv | awk '{print $5,$9}'

2.5G /data/merged_csv/BitMEX.Swap.BTC_USD.csv
2.1G /data/merged_csv/Binance.Spot.BTC_USDT.csv
2.0G /data/merged_csv/Binance.Swap.BTC_USDT.csv
1.9G /data/merged_csv/Huobi.Spot.BTC_USDT.csv
1.7G /data/merged_csv/Huobi.Swap.BTC_USD.csv
1.5G /data/merged_csv/OKEx.Spot.BTC_USDT.csv
860M /data/merged_csv/OKEx.Swap.BTC_USD.csv
515M /data/merged_csv/OKEx.Swap.BTC_USDT.csv
386M /data/merged_csv/Bitfinex.Spot.BTC_USD.csv
349M /data/merged_csv/CoinbasePro.Spot.BTC_USD.csv
175M /data/merged_csv/WhaleEx.Spot.BTC_USDT.csv
112M /data/merged_csv/Bitfinex.Spot.BTC_USDT.csv
103M /data/merged_csv/MXC.Spot.BTC_USDT.csv
83M /data/merged_csv/Bitstamp.Spot.BTC_USD.csv
72M /data/merged_csv/Kraken.Spot.BTC_USD.csv
5.9M /data/merged_csv/Bitfinex.Swap.BTC_USDT.csv
3.2M /data/merged_csv/Kraken.Spot.BTC_USDT.csv
5.4K /data/merged_csv/Newdex.Spot.BTC_USDT.csv


In [33]:
!ls -lhS /data/merged_csv/*ETH_USD*.csv | awk '{print $5,$9}'

1013M /data/merged_csv/Huobi.Swap.ETH_USD.csv
932M /data/merged_csv/Huobi.Spot.ETH_USDT.csv
636M /data/merged_csv/Binance.Swap.ETH_USDT.csv
575M /data/merged_csv/OKEx.Spot.ETH_USDT.csv
564M /data/merged_csv/Binance.Spot.ETH_USDT.csv
369M /data/merged_csv/BitMEX.Swap.ETH_USD.csv
300M /data/merged_csv/OKEx.Swap.ETH_USD.csv
149M /data/merged_csv/OKEx.Swap.ETH_USDT.csv
122M /data/merged_csv/MXC.Spot.ETH_USDT.csv
118M /data/merged_csv/CoinbasePro.Spot.ETH_USD.csv
58M /data/merged_csv/WhaleEx.Spot.ETH_USDT.csv
57M /data/merged_csv/Bitfinex.Spot.ETH_USD.csv
32M /data/merged_csv/Kraken.Spot.ETH_USD.csv
19M /data/merged_csv/Bitstamp.Spot.ETH_USD.csv
11M /data/merged_csv/Bitfinex.Spot.ETH_USDT.csv
3.1M /data/merged_csv/Bitfinex.Swap.ETH_USDT.csv
1.3M /data/merged_csv/Kraken.Spot.ETH_USDT.csv
5.9K /data/merged_csv/Newdex.Spot.ETH_USDT.csv


In [29]:
tasks = generate_tasks(
    btc_files + eth_files,
    'TimeBar',
    [10000, 60000, 180000, 300000, 900000, 1800000, 3600000],
    '/data/bars/TimeBar',
) + generate_tasks(
    btc_files + eth_files,
    'TickBar',
    [4, 8, 16, 32, 64, 128],
    '/data/bars/TickBar',
) + generate_tasks(
    btc_files,
    'VolumeBar',
    [1, 2, 4, 8, 16, 32],
    '/data/bars/VolumeBar',
) + generate_tasks(
    eth_files,
    'VolumeBar',
    [10, 20, 40, 80, 160, 320],
    '/data/bars/VolumeBar',
) + generate_tasks(
    btc_files,
    'DollarBar',
    [10000, 20000, 40000, 80000, 160000, 320000],
    '/data/bars/DollarBar',
) + generate_tasks(
    eth_files,
    'DollarBar',
    [2000, 4000, 8000, 16000, 32000],
    '/data/bars/DollarBar',
)

In [30]:
len(tasks)

882

In [31]:
run_tasks_parallell(tasks)

[########################################] | 100% Completed |  1hr 40min 53.6s


## References

* [Tick, Volume, Dollar Volume Bars.ipynb](https://github.com/BlackArbsCEO/Adv_Fin_ML_Exercises/blob/master/notebooks/Tick%2C%20Volume%2C%20Dollar%20Volume%20Bars.ipynb)